# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert 🔠 words to 🔢 vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>

_Embeddings_ are representations of words using vectors. These embeddings can be learned within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representations into a Recurrent Neural Network. 

▶️ Run this cell and make sure the version of 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) you are using is ≥ 4.0!

In [1]:
!pip freeze | grep gensim

gensim==4.2.0


In [2]:
!pip freeze | grep tensorflow

tensorflow==2.10.0
tensorflow-datasets==4.6.0
tensorflow-estimator==2.10.0
tensorflow-io-gcs-filesystem==0.27.0
tensorflow-metadata==1.10.0


# The data

Keras provides many datasets, among which is the IMDB dataset 🎬:
- It is comprised of sentences that are ***movie reviews***. 
- Each of these reviews is related to a score given by the reviewer.

❓ **Question** ❓ First of all, let's load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer can handle it. Otherwise, rerun with a lower number.  

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [3]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2023-05-29 10:19:13.624122: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-29 10:19:14.183649: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-29 10:19:14.306778: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-29 10:19:14.306798: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

<b><u>Embeddings in the previous challenge</u></b>:

In the previous exercise, we jointly learned a representation for the words, and fed this representation to a RNN, as shown down below 👇: 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which slows down and increases the difficulty of convergence!

<b><u>Embeddings in the current challenge</u></b>:

For this reason, we will separate the steps of learning the word representation and feeding it into a RNN. As shown here: 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with Word2Vec.

The drawback is indeed that the learned embeddings are not _specifically_ designed for our task. However, learning them independently of the task at hand (sentiment analysis) has some advantages: 
- it is very fast to do in general (with Word2Vec)
- the representation learned by Word2Vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with Word2Vec and see how meaningful it is!

# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 🐶 _dog_ $\rightarrow$ [0.1, -0.3, 0.8]
- 🐱 _cat_ $\rightarrow$ [-1.1, 2.3, 0.7]
- 🍏 _apple_ $\rightarrow$ [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

***What is a "good" numerical representation of words?***

- ***Words with close meanings should be geometrically close in your embedding space!***

    - Look at the following example which represents a bi-dimensional embedding space.

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! 

[📚 **Gensim**](https://radimrehurek.com/gensim/)  is a great Python package that makes the use of the Word2Vec algorithm easy to implement, fast and accurate (which is not an easy task!).

1. The following code imports Word2Vec from Gensim. 

2. The second line learns the embedding representation of the words thanks to the sentences in `X_train`. 
3. The third line stores the words and their trained embeddings in `wv`.

In [14]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)
wv = word2vec.wv

Let's look at the embedded representation of some words.

You can use `wv` as a dictionary.
For instance, `wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation was not learned). 

In [15]:
# YOUR CODE HERE
wv["dog"]

array([-2.08366796e-01,  1.87758103e-01, -9.34619009e-02,  1.95202723e-01,
       -1.98869947e-02, -3.89202952e-01,  2.62203477e-02,  5.67088366e-01,
       -1.98280394e-01, -2.17946246e-01, -1.46437138e-01, -3.39779884e-01,
       -4.45826678e-03,  1.19523004e-01, -4.44210805e-02, -2.28080183e-01,
        1.61739305e-01, -3.01454723e-01, -1.65856499e-02, -4.06961650e-01,
        1.62342876e-01,  7.72206485e-02,  3.04929227e-01, -9.92648005e-02,
        4.10524458e-02,  9.12184734e-03, -2.55638599e-01, -9.96744186e-02,
       -1.51754841e-01,  4.63533327e-02,  1.90860450e-01,  1.53818735e-04,
        1.67210907e-01, -3.76329780e-01, -8.03921297e-02,  2.44672060e-01,
        7.09967911e-02, -1.50895789e-01, -2.06129372e-01, -3.11419159e-01,
       -2.23062947e-01, -3.17592472e-01, -2.54699409e-01,  3.07497889e-01,
        3.20263386e-01, -3.08806952e-02, -1.81695446e-01, -1.06524855e-01,
        1.96052432e-01,  1.52389586e-01,  9.36763361e-02, -1.61186039e-01,
       -2.54136413e-01,  

In [16]:
wv["cat"]

array([-0.15731803,  0.3293582 , -0.29441127,  0.3013499 , -0.08752959,
       -0.39023298,  0.1845021 ,  0.84843343, -0.19159067, -0.3108013 ,
       -0.0224323 , -0.51191556,  0.1392684 ,  0.19593138, -0.09823272,
       -0.26947528,  0.32318255, -0.30432522,  0.04526765, -0.39607635,
        0.23492607,  0.01394091,  0.206723  , -0.31337845, -0.03141876,
        0.03674737, -0.3303583 , -0.18160369, -0.3223909 , -0.00114467,
        0.16251683, -0.01903086,  0.12398297, -0.5549864 , -0.19492516,
        0.20271446,  0.14367639, -0.19910634, -0.42531353, -0.19474348,
       -0.24348514, -0.35513052, -0.3006355 ,  0.2543732 ,  0.43927005,
       -0.13052566, -0.17822784, -0.06292172,  0.51276165,  0.21514793,
        0.15212245, -0.19280863, -0.19378786,  0.04751308, -0.33625284,
        0.20476468,  0.27352723, -0.11324871, -0.06405751,  0.06123474,
        0.29025018, -0.10710628, -0.02645814,  0.3212258 , -0.08729419,
        0.16960637, -0.1100309 ,  0.309366  , -0.33827358,  0.35

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [17]:
# YOUR CODE HERE
wv["dog"].size

100

🧐 How do we know whether this embedding make any sense or not? 

💡 To investigate this question, we will check that words with a close meaning have close representations. 

👉 Let's use the [**`Word2Vec.wv.most_similar`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar) method that, given an input word, displays the "closest" words in the embedding space. If the embedding is well done, then words with similar meanings will have similar representation in the embedding space.

❓ **Question** ❓ Try out the `most_similar` method on different words. 

🧑🏿‍🏫 The quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [27]:
# YOUR CODE HERE
wv.most_similar("dog", topn=5)

[('mel', 0.9843147993087769),
 ('constantly', 0.983116090297699),
 ('adam', 0.9827868938446045),
 ('ms', 0.9822023510932922),
 ('driver', 0.9815793633460999)]

In [28]:
wv.most_similar("cat", topn=5)

[('boss', 0.9888152480125427),
 ('mate', 0.9857512712478638),
 ('door', 0.9841458201408386),
 ('apartment', 0.9839611649513245),
 ('trial', 0.9825825095176697)]

📚 Similarly to `most_similar` used on words directly, we can use [**`similar_by_vector`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similar_by_vector) on vectors to do the same thing:

In [29]:
# YOUR CODE HERE

wv.similar_by_vector("dog", topn=5)

[('mel', 0.9843147993087769),
 ('constantly', 0.983116090297699),
 ('adam', 0.9827868938446045),
 ('ms', 0.9822023510932922),
 ('driver', 0.9815793633460999)]

# Arithmetic on words

Now, let's perform some mathematical operations on words, i.e. on their vector representations!

As any learned word is represented as a vector, you can do basic arithmetic operations, such as:

$$W2V(good) - W2V(bad)$$

In [40]:
word2vec.wv.key_to_index

{'the': 0,
 'a': 1,
 'and': 2,
 'of': 3,
 'to': 4,
 'is': 5,
 'br': 6,
 'in': 7,
 'i': 8,
 'it': 9,
 'this': 10,
 'that': 11,
 'was': 12,
 'as': 13,
 'for': 14,
 'with': 15,
 'but': 16,
 'movie': 17,
 'film': 18,
 'on': 19,
 'not': 20,
 'you': 21,
 'his': 22,
 'are': 23,
 'have': 24,
 'one': 25,
 'be': 26,
 'he': 27,
 'all': 28,
 'at': 29,
 'by': 30,
 'they': 31,
 'an': 32,
 'so': 33,
 'like': 34,
 'who': 35,
 'from': 36,
 'her': 37,
 'or': 38,
 'just': 39,
 'if': 40,
 'out': 41,
 'about': 42,
 "it's": 43,
 'has': 44,
 'what': 45,
 'some': 46,
 'there': 47,
 'good': 48,
 'more': 49,
 'when': 50,
 'very': 51,
 'no': 52,
 'up': 53,
 'she': 54,
 'my': 55,
 'time': 56,
 'even': 57,
 'which': 58,
 'would': 59,
 'really': 60,
 'only': 61,
 'had': 62,
 'story': 63,
 'me': 64,
 'see': 65,
 'can': 66,
 'their': 67,
 'were': 68,
 'well': 69,
 'than': 70,
 'much': 71,
 'get': 72,
 'do': 73,
 'great': 74,
 'been': 75,
 'we': 76,
 'first': 77,
 'bad': 78,
 'because': 79,
 'into': 80,
 'other': 81,


❓ **Question** ❓ Do this mathematical operation and print the result

In [44]:
# YOUR CODE HERE
v_good = word2vec.wv["good"]
v_bad = word2vec.wv["bad"]

v_result = v_good - v_bad
word2vec.wv.similar_by_vector(v_result)

[('who', 0.42709290981292725),
 ('young', 0.3194664716720581),
 ('been', 0.25670528411865234),
 ('by', 0.23368579149246216),
 ('his', 0.20787926018238068),
 ('hailed', 0.20463088154792786),
 ('dahl', 0.2044321596622467),
 ('visited', 0.20420211553573608),
 ('woman', 0.19884811341762543),
 ('an', 0.19751796126365662)]

Now, imagine for a second that the following equality holds true:

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to:

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be bold of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [45]:
# YOUR CODE HERE
v_good = word2vec.wv["good"]
v_bad = word2vec.wv["bad"]
v_stupid = word2vec.wv["stupid"]

res = v_good - v_bad + v_stupid
word2vec.wv.similar_by_vector(res)

[('good', 0.8482003808021545),
 ('nice', 0.8240365386009216),
 ('great', 0.8107876181602478),
 ('decent', 0.7769801616668701),
 ('always', 0.7653427720069885),
 ('also', 0.7414466738700867),
 ('colin', 0.7294730544090271),
 ('such', 0.7226190567016602),
 ('potential', 0.7218870520591736),
 ('groundbreaking', 0.7191649675369263)]

We said earlier, that for any vector it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vectors of `res`

💡 _Hint_: `similar_by_vector`

In [52]:
# YOUR CODE HERE
wv.similar_by_vector(res, topn=5)

[('good', 0.8482003808021545),
 ('nice', 0.8240365386009216),
 ('great', 0.8107876181602478),
 ('decent', 0.7769801616668701),
 ('always', 0.7653427720069885)]

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [56]:
v_queen = wv["queen"]
v_king = wv["king"]
v_actor = wv["actor"]

res = v_queen - v_king + v_actor
wv.similar_by_vector(res)

[('actor', 0.9788129329681396),
 ('actress', 0.8897715210914612),
 ('performance', 0.8766772150993347),
 ('role', 0.8532341718673706),
 ('character', 0.8475448489189148),
 ('job', 0.8090875148773193),
 ('conscience', 0.8008187413215637),
 ('billed', 0.7932430505752563),
 ('guy', 0.7883970737457275),
 ('duke', 0.7803231477737427)]

In [55]:
# YOUR CODE HERE
v_boy = wv["boy"]
v_girl = wv["girl"]
v_woman = wv["woman"]

res = v_boy - v_girl + v_woman
wv.similar_by_vector(res)

[('boy', 0.9714475870132446),
 ('woman', 0.9645140171051025),
 ('child', 0.9485822916030884),
 ('servant', 0.9464004039764404),
 ('husband', 0.9457123875617981),
 ('affair', 0.9450232982635498),
 ('friend', 0.9424946308135986),
 ('mother', 0.9419435858726501),
 ('married', 0.9364405274391174),
 ('son', 0.9339624643325806)]

<u><i>Some notes about Word2Vec as an internal Neural Network</i></u>:

You might wonder where does this magic comes from (at quite a low price, you just ran a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)`, actually trains a internal neural network (that you don't see).  

In a nutshell, this internal neural network predicts a word from the surroundings words in a sentences. Hence, it splits the original sentences, then for each split it chooses some words as inputs $X$ and a word as the output $y$ which it tries to predict, using the embedding space.

And as with any neural network, Word2Vec has some hyperparameters. Let's play with some of these. 

# Word2Vec hyperparameters

❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the embedding size is the one you chose.

In [62]:
# YOUR CODE HERE
word2vec_2 = Word2Vec(sentences=X_train, vector_size=10)
wv = word2vec_2.wv

In [64]:
wv["life"].size

10

❓ **Question** ❓ Use the **`Word2Vec.wv.key_to_index`** attribute to display the size of the learned vocabulary. Compare it to the number of different words in `X_train`.

In [67]:
# YOUR CODE HERE
print(len(wv.key_to_index))
print(len(X_train))

8006
2500


There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been trained on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec:  `min_count`. 

`min_count` is a integer that tells you how many occurrences a given word should have to be learned in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to learn a representation of words that are sufficiently present in the corpus to have a robust embedded representation.

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vecs that you have trained (you can choose any `vector_size` you want).

In [69]:
# YOUR CODE HERE
word2vec_3 = Word2Vec(sentences=X_train, vector_size=10, min_count=5)
wv = word2vec_3.wv
len(wv.key_to_index)

8006

In [70]:
word2vec_4 = Word2Vec(sentences=X_train, vector_size=10, min_count=4)
wv = word2vec_4.wv
len(wv.key_to_index)

9584

Remember that Word2Vec has an internal neural network that is optimized based on some predictions. These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the Word2Vec with different `window` sizes.

❓ **Question** ❓ Train a new `word2vec_5` model with a `window` different than previously (default is 5).

In [71]:
# YOUR CODE HERE
word2vec_5 = Word2Vec(sentences=X_train, vector_size=10, min_count=5, window=6)
wv = word2vec_5.wv

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the ones that you should start playing with to get a better performance for your model.

But you can also look at other arguments in the [**📚 Documentation - gensim.models.word2vec.Text8Corpus**](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)

# Convert our train and test set to RNN-ready datasets

Remember that `Word2Vec` is the first step to the overall process of feeding such a representation into a RNN, as shown here:

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be fed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (make sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- For the <font color=orange>test</font> set, this is understandable: <font color=orange>some words were not</font> in the <font color=blue>train</font> set and thus, their <font color=orange>embedded representation is unknown</font>
- for the <font color=blue>train set</font>, due to `min_count` hyperparameter, not all the words have a vector representation.

In any case, just skip the missing words here.

In [80]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']

def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
    
    return np.array(embedded_sentence)
### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)  
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

❓ **Question** ❓ Write a function that, given a list of sentences (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

💡 _Hint_: Use the previous function `embed_sentence`

In [81]:
def embedding(word2vec, sentences):
    embedding = []
    for sentence in sentences:
        embedded_sentences = embed_sentence(word2vec, sentence)
        embedding.append(embedded_sentences)
        
        return embedding
        
X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad your sequences so you have tensors which can be divided into batches (of `batch_size`) during the optimization. Store the padded values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [82]:
### YOUR CODE HERE

X_train_pad = pad_sequences(X_train, dtype="float", padding="post")
X_test_pad = pad_sequences(X_test, dtype="float", padding="post")

assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)



🏁 Congratulations, you are now able to use `Word2Vec` to embed your words :)

💾 Don't forget to git add/commit/push your notebook...

🚀 ... and move on to the next challenge!
